# LangChain OpenAI Assistant agent

In [ ]:
! pip3 install openai
! pip3 install langchain

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [2]:
from agentquest.drivers.mastermind import MasterMindDriver
from agentquest.utils import Action, load_data
from termcolor import cprint

Load a generic 4 digits mastermind game

In [3]:
game = load_data('mastermind', '4 digits')[0]

Initialize the prompt and the OpenAI Assistant agent

In [4]:
from langchain.agents.openai_assistant import OpenAIAssistantRunnable
from langchain.schema.agent import AgentFinish
from langchain.tools.base import BaseTool

prompt = """
You are tasked to play the Mastermind game. The host chooses a number and gives
you the amount of digits. You have to gess the correct number as fast as you can.

Rules:
1. At each round you provide a number as guess. The hosts provides you the number
of correct digits in the wrong position and the number of correct digits in the
correct position.
2. The game ends when the hosts outputs 'You Won!'
3. The number can contain repetitions.
4. The number can contain any possible digit.
5. Avoid bruteforce.

Guess the number:
"""

# Define a dummy tool for closed-box environments
class ProxyTool(BaseTool):
    name = "proxytool"
    description = "Provide the action you want to perform"
    def _run(self): 
        pass

In [5]:
agent = OpenAIAssistantRunnable.create_assistant(
    name="Mastermind OpenAI assistant",
    instructions=prompt,
    tools=[ProxyTool()],
    model="gpt-4-1106-preview",
    as_agent=True,
)

Initialize the mastermind driver and get the first observation

In [6]:
driver = MasterMindDriver(game)
obs = driver.reset() # Get the first observation
print(f'OBSERVATION: {obs.output}')

OBSERVATION: Start guessing the 4 digits number.


Start the Agent loop solving the task

In [7]:
response = agent.invoke({"content": obs.output})
step_cnt = 0
while not obs.done and step_cnt<10 and not isinstance(response, AgentFinish):
    tool_outputs = []
    # print(response)
    for _action in response:

        agent_guess = _action.tool_input
        # print(_action)
        action = Action(action_value=agent_guess)
        print(agent_guess)
        # Build the action and perform the step
        obs = driver.step(action)
        print(obs.output)
        if obs.done:
            break
        # Manage Proxy Tool output
        tool_outputs.append(
            {"output": obs.output, 
            "tool_call_id": _action.tool_call_id}
        )
    # Get the new action
    response = agent.invoke(
        {"tool_outputs": tool_outputs,
        "run_id": _action.run_id,
        "thread_id": _action.thread_id}
    )

    step_cnt+=1

1234
Your guess has 1 correct numbers in the wrong position and 0 correct numbers in the correct position. Keep guessing.
5678
Your guess has 0 correct numbers in the wrong position and 2 correct numbers in the correct position. Keep guessing.
5876
Your guess has 1 correct numbers in the wrong position and 1 correct numbers in the correct position. Keep guessing.
5676
Your guess has 0 correct numbers in the wrong position and 1 correct numbers in the correct position. Keep guessing.
5768
Your guess has 0 correct numbers in the wrong position and 2 correct numbers in the correct position. Keep guessing.
5798
Your guess has 1 correct numbers in the wrong position and 2 correct numbers in the correct position. Keep guessing.
5978
Your guess has 0 correct numbers in the wrong position and 3 correct numbers in the correct position. Keep guessing.
5970
Your guess has 0 correct numbers in the wrong position and 2 correct numbers in the correct position. Keep guessing.
5971
Your guess has 1 co